# Notebook #1


### Name: Tulsi Patel


### Date: 11.11.2021


### PID: 730392259


### Goal: Find average firing rate for Brain Observatory, Sst, Male, VISp, Static Gratings (SG) and Drifting Gratings (DG). 

*There are no Female SST VISl sessions. The Female SST VISp sessions have 0 low-contamination units. There are no Female WT/WT VISp and VISl sessions. (Disregard the Jupyter notebooks labeled "Female WT". They are not actually WT/WT. They are hybrids. 

## Protocol

### Starting code. 

In [1]:
import os

import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys.ecephys_session import (
    EcephysSession
)
from allensdk.brain_observatory.ecephys.visualization import plot_mean_waveforms, plot_spike_counts, raster_plot
from allensdk.brain_observatory.visualization import plot_running_speed

# tell pandas to show all columns when we display a DataFrame
pd.set_option("display.max_columns", None)

In [2]:
data_directory = '/Users/tulsipatel/local1/ecephys_cache_dir' 
# must be updated to a valid directory in your filesystem

manifest_path = os.path.join(data_directory, "manifest.json")

In [3]:
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

In [4]:
units = cache.get_units()

In [5]:
sessions = cache.get_session_table()


print('Total number of sessions: ' + str(len(sessions)))

sessions.head()

Total number of sessions: 58


,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
721123822,2019-10-03T00:00:00Z,707296982,brain_observatory_1.1,125.0,M,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,444,2229,6,"[MB, SCig, PPT, NOT, DG, CA1, VISam, nan, LP, ..."
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"


In [6]:
filtered_sessions = sessions[(sessions.sex == 'M') & \
                             (sessions.full_genotype.str.find('Sst') > -1) & \
                             (sessions.session_type == 'brain_observatory_1.1') & \
                             (['VISp' in acronyms for acronyms in 
                               sessions.ecephys_structure_acronyms])]

filtered_sessions.head()

,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
756029989,2019-10-03T00:00:00Z,734865738,brain_observatory_1.1,96.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,684,2214,6,"[TH, DG, CA3, CA1, VISl, nan, PO, Eth, LP, VIS..."
758798717,2019-10-03T00:00:00Z,735109609,brain_observatory_1.1,102.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,593,1486,4,"[MGv, MGd, TH, DG, CA1, VISal, nan, SGN, LP, S..."
762602078,2019-10-03T00:00:00Z,744915204,brain_observatory_1.1,110.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,531,2233,6,"[MB, APN, NOT, DG, CA1, VISam, nan, VISrl, LGv..."


In [7]:
session_id = filtered_sessions.index.values[1]

session = cache.get_session_data(session_id)

Downloading:   0%|          | 0.00/3.07G [00:00<?, ?B/s]

In [8]:
units = session.units

print('Total number of units: ' + str(len(units)))

units.head()

/Users/tulsipatel/.local/lib/python3.8/site-packages/allensdk/brain_observatory/ecephys/ecephys_project_api/ecephys_project_warehouse_api.py:291: FutureWarning: Conversion of the second argument of issubdtype from `bool` to `np.generic` is deprecated. In future, it will be treated as `np.bool_ == np.dtype(bool).type`.
  pv_is_bool = np.issubdtype(output["p_value_rf"].values[0], np.bool)


Total number of units: 755


,waveform_duration,cluster_id,peak_channel_id,cumulative_drift,amplitude_cutoff,snr,waveform_recovery_slope,isolation_distance,nn_miss_rate,silhouette_score,waveform_velocity_above,waveform_PT_ratio,L_ratio,waveform_velocity_below,max_drift,isi_violations,firing_rate,waveform_amplitude,local_index_unit,waveform_spread,waveform_halfwidth,d_prime,presence_ratio,waveform_repolarization_slope,nn_hit_rate,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
950916730,0.315913,0,850252273,421.99,0.010911,2.408535,-0.072013,115.066739,0.001010,NaN,1.442211,0.478356,0.000007,NaN,21.73,0.005433,9.997023,84.680895,0,40.0,0.178559,7.166834,0.99,0.363023,1.000000,NaN,200.0,3.361502,1.292960,0.702449,1.364103,0.862268,0.268027,0.029584,0.026160,0.037801,85.000,0.361121,0.203,0.08,2.0,-0.500553,-0.639423,-0.438080,NaN,-0.529617,0.0,120.0,0.000007,0.000002,0.000129,NaN,6.919468e-07,50.000,5015,0.75,9.050915,4.902551,12.940972,1.266454,12.912920,NaN,0.0055,False,False,False,False,False,0.078169,False,0.029950,0.051790,0.006392,0.235197,0.008718,60,43,729445648,4,215.0,APN,8369.0,3658.0,6985.0,probeA,See electrode locations,29999.967418,1249.998642,True
950916822,0.233501,7,850252319,294.57,0.000025,3.769478,-0.212549,160.983322,0.000000,0.115187,0.000000,0.565649,0.002818,-0.137353,79.83,0.031287,34.835261,217.711650,7,80.0,0.192295,6.721923,0.99,0.988637,0.998667,NaN,200.0,7.543934,1.444267,3.117428,2.664432,1.825410,0.172680,0.016890,0.023038,0.026824,90.000,1.283263,0.524,0.08,2.0,-0.272234,-0.263533,-0.273504,-0.230975,-0.171521,135.0,90.0,0.219674,0.042044,0.026586,0.314268,1.054968e-01,5.000,5013,0.50,33.983293,30.081415,35.384484,33.720582,35.828590,NaN,0.0085,False,False,False,False,False,0.249576,False,0.004937,0.023460,0.002753,0.014044,0.004153,280,27,729445648,27,215.0,APN,8288.0,3464.0,7039.0,probeA,See electrode locations,29999.967418,1249.998642,True
950916845,0.192295,9,850252323,270.88,0.023245,3.087942,-0.216281,77.035531,0.002380,0.170339,-0.480737,0.683272,0.003132,0.000000,77.64,0.376063,4.293337,156.579540,9,80.0,0.137353,4.525955,0.99,0.685330,0.955088,NaN,600.0,21.424762,2.213333,5.706429,7.988889,4.428571,0.861313,0.041113,0.083601,0.184850,81.667,0.212391,0.393,0.08,2.0,-0.755208,-0.726496,-0.884910,-0.706667,-0.570116,315.0,30.0,0.144120,0.012421,0.000508,0.338912,6.371648e-02,43.333,4956,0.50,3.916450,2.477920,5.485774,2.870630,4.643349,NaN,0.0115,False,False,False,False,False,0.153542,False,0.063920,0.011267,0.042185,0.213324,0.055352,300,11,729445648,29,215.0,APN,8281.0,3448.0,7044.0,probeA,See electrode locations,29999.967418,1249.998642,True
950916856,0.233501,10,850252325,219.19,0.011552,2.743504,-0.087931,291.708582,0.047256,0.141643,0.824121,0.380183,0.018622,0.156975,71.95,0.022541,31.345340,120.485235,10,90.0,0.123618,4.440113,0.99,0.511236,0.946000,NaN,200.0,4.767794,1.014055,1.564724,2.439583,1.028398,0.186595,0.026503,0.020562,0.015143,90.000,1.255062,0.500,0.08,2.0,-0.268519,-0.275178,-0.14068

In [9]:
V1_units = units[(units.ecephys_structure_acronym == 'VISp') &
                (units.isi_violations < 0.1)]

print('Total number of low-contamination V1 units: ' + str(len(V1_units)))


V1_units.head()

Total number of low-contamination V1 units: 40


,waveform_duration,cluster_id,peak_channel_id,cumulative_drift,amplitude_cutoff,snr,waveform_recovery_slope,isolation_distance,nn_miss_rate,silhouette_score,waveform_velocity_above,waveform_PT_ratio,L_ratio,waveform_velocity_below,max_drift,isi_violations,firing_rate,waveform_amplitude,local_index_unit,waveform_spread,waveform_halfwidth,d_prime,presence_ratio,waveform_repolarization_slope,nn_hit_rate,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
950934506,0.549414,330,850251307,959.66,0.071696,1.974889,-0.007850,94.000936,0.000285,0.079577,NaN,3.739630,0.000495,NaN,47.12,0.083688,3.119496,57.233280,320,10.0,0.164824,4.721787,0.99,0.116794,0.988000,NaN,400.0,7.365401,0.785369,3.411429,2.558333,3.124641,0.935157,0.346929,0.441347,0.184960,17.500,7.749461,0.320,0.02,8.0,-0.121429,0.455274,0.247088,0.607143,-0.159901,225.0,60.0,0.757282,1.770280e-01,0.185696,0.260151,0.361799,12.500,5022,0.00,0.990135,5.861746,5.161343,1.369525,5.317437,NaN,0.0725,False,False,False,False,False,0.742729,False,0.371946,0.001486,0.175814,0.262924,0.265392,2520,59,729445652,250,385.0,VISp,8434.0,1249.0,7661.0,probeC,See electrode locations,29999.997521,1249.999897,True
950934565,0.879062,336,850251321,139.52,0.000468,2.959099,-0.025951,66.252958,0.000239,0.171097,1.922948,0.380742,0.001132,-0.206030,44.07,0.044149,2.095701,88.130640,326,90.0,0.192295,5.739527,0.99,0.240532,0.969419,NaN,300.0,4.757685,0.807426,1.438750,1.066667,1.548768,0.288984,0.031911,0.800756,0.554087,90.000,0.171182,0.000,0.04,2.0,0.485190,-0.048951,-0.261905,0.714286,0.362587,135.0,150.0,0.033603,9.187110e-01,0.328015,0.023111,0.001391,10.000,4966,0.75,3.163757,2.397987,1.360864,2.593216,3.982583,NaN,0.1095,False,False,False,False,False,0.591898,False,0.633764,0.726815,0.318062,0.131798,0.504525,2580,11,729445652,257,385.0,VISp,8451.0,1184.0,7704.0,probeC,See electrode locations,29999.997521,1249.999897,True
950934544,0.604355,334,850251319,179.04,0.004833,1.768953,-0.018645,87.270236,0.001122,0.116079,0.412060,0.365980,0.001838,0.206030,55.40,0.003252,5.981363,54.273180,324,80.0,0.302178,4.247322,0.99,0.177085,0.979333,NaN,1400.0,1.489947,1.038333,0.559412,0.784496,1.061439,0.489484,0.041613,0.097330,0.031466,68.571,0.505982,0.015,0.08,8.0,-0.068750,0.012000,-0.232749,-0.533333,-0.478390,180.0,120.0,0.710812,9.662706e-01,0.077220,0.021092,0.003885,-20.714,4929,0.25,5.074440,6.554498,7.069642,6.016481,5.817675,NaN,0.1215,False,False,False,False,False,0.103475,False,0.069282,0.056647,0.030082,0.057366,0.019744,2580,43,729445652,256,385.0,VISp,8448.0,1193.0,7698.0,probeC,See electrode locations,29999.997521,1249.999897,True
950934614,0.741709,340,850251323,120.74,0.000006,3.393787,-0.014714,60.207322,0.001637,0.080105,0.000000,0.304352,0.010162,-1.098827,46.25,0.008980,4.156224,123.122805,330,60.0,0.192295,4.916224,0.99,0.409432,0.933642,NaN,1000.0,0.959879,0.616667,0.448785,0.347619,0.386557,0.182507,0.019267,0.381476,0.423002,24.000,1.189384,0.000,0.08,15.0,0.0953

## Find average firing rate for static gratings. 

In [10]:
print(V1_units.firing_rate_sg)

unit_id
950934506     5.317437
950934565     3.982583
950934544     5.817675
950934614     7.130548
950934597    10.438377
950934762    59.225847
950934750    10.975915
950934786     6.083447
950934912     3.405078
950934987    29.641233
950935081     7.245783
950935066    25.140428
950935168     5.475302
950935043     9.297356
950935027    10.489000
950935142     1.563325
950935154    18.540091
950936618     7.436952
950935287    11.243019
950935276    15.115030
950936593     5.405362
950935264     8.393465
950935244     9.049568
950935366     4.237697
950935354     3.644872
950936606     0.933865
950935320     6.302592
950935416    17.576918
950935440     6.032157
950935403     1.723188
950935552    12.981528
950935529     8.268905
950935480    12.227508
950935465     3.289844
950935587     6.670277
950935726     1.644589
950935804    12.906259
950935840     9.738311
950936810     0.112570
950935909     3.884001
Name: firing_rate_sg, dtype: float64


In [11]:
static_firing_rates=V1_units.firing_rate_sg 
static_average_firing_rate = sum(V1_units.firing_rate_sg)/len(V1_units.firing_rate_sg)
print(static_average_firing_rate)

9.714697539962184


## Find average firing rate for drifting gratings. 

In [12]:
print(V1_units.firing_rate_dg)

unit_id
950934506     0.990135
950934565     3.163757
950934544     5.074440
950934614     8.610560
950934597     7.520031
950934762    29.555839
950934750     3.569584
950934786     7.430260
950934912     1.904841
950934987    34.082094
950935081     3.514341
950935066     6.825236
950935168     2.568295
950935043    14.759063
950935027     8.557441
950935142     2.195932
950935154    13.900131
950936618     1.554257
950935287     6.534677
950935276     9.083848
950936593     3.650325
950935264     5.486111
950935244     3.081954
950935366     3.652981
950935354     0.934360
950936606     0.737820
950935320     2.580512
950935416     9.089160
950935440    12.468579
950935403     1.756639
950935552     9.347318
950935529     5.920092
950935480     7.771814
950935465     2.122097
950935587     1.523979
950935726     0.488162
950935804     9.394062
950935840    10.660946
950936810     0.066399
950935909     6.688190
Name: firing_rate_dg, dtype: float64


In [13]:
drifting_firing_rates=V1_units.firing_rate_dg 
drifting_average_firing_rate = sum(V1_units.firing_rate_dg)/len(V1_units.firing_rate_dg)
print(drifting_average_firing_rate)

6.720406449757876


## Remaining code: I was experimenting with another way to find the average firing rate. 

In [ ]:
presentation_table = session.stimulus_presentations[session.stimulus_presentations.stimulus_name == 'natural_scenes']


presentation_times = presentation_table.start_time.values
presentation_ids = presentation_table.index.values
presentation_table.head()

In [ ]:
units_of_interest = V1_units.index.values

In [ ]:
times = session.presentationwise_spike_times(
    stimulus_presentation_ids=presentation_ids,
    unit_ids=units_of_interest
)

times.head()

In [ ]:
average_firing_rate = '.0023476'

In [ ]:
length_of_time=times.time_since_stimulus_presentation_onset

In [ ]:
print(length_of_time.index.values[4])